<H1>ANKH</H1>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys

sys.path.insert(0, os.path.abspath('../../../pyphetools'))
from pyphetools import *

<h3>Import HPO Dara</h3>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()

<H1>Importing a single case report</H1>
<p>Here, we use functions of the pyphetools package to import data from a typical case report: <a href="https://pubmed.ncbi.nlm.nih.gov/33748234/" target="__blank">Wu JL, et al.</a> A three-year clinical investigation of a Chinese child with craniometaphyseal dysplasia caused by a mutated ANKH gene. World J Clin Cases. 2021 Mar 16;9(8):1853-1862.</p>
<p>The case report consists of several sections to which we can apply text mining and add some corrections for cases in which text mining fails to capture an HPO term or calls a false-positive term.</p>

In [3]:
pmid = "PMID:33748234"
age = "P1Y5M"
parser = CaseParser(concept_recognizer=hpo_cr, pmid=pmid, age_at_last_exam=age)

<h3>Chief complaints</h3>
<p>A 17-mo-old boy presented with progressive nasal obstruction, snoring and hearing loss symptoms when referred to the hospital.</p>

In [4]:
vignette = "A 17-mo-old boy presented with progressive nasal obstruction, snoring and hearing loss symptoms when referred to the hospital."
results = parser.add_vignette(vignette=vignette)

In [5]:
results

,id,label,observed,measured
0,HP:0000365,Hearing impairment,True,True
1,HP:0001742,Nasal congestion,True,True
2,HP:0025267,Snoring,True,True
3,HP:0500001,Body odor,True,True
